# Agentic Workflows
> Self Refine

![](https://raw.githubusercontent.com/madaan/self-refine/main/docs/static/images/animation_oldstyle_oneloop.gif)

[paper](https://arxiv.org/pdf/2303.17651.pdf)

[Hemanth HM](https://h3manth.com)

# Install Deps

In [ ]:
!pip install -q -U langchain
!pip install --quiet langchain-google-genai

## Helper Functions

In [44]:
import pathlib
import textwrap


from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

## Set up the Keys

In [12]:
from google.colab import userdata
import os
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

## *Instantiate* gemini-pro



In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-pro")

# Prompts

In [7]:
from langchain import PromptTemplate

In [59]:
programmer = PromptTemplate.from_template("""
Prompt:
You are an AI code generator trained on a massive dataset of code and capable of producing high-quality, functional code in various programming languages.
Your task is to write code that performs the following function:
{Task}
Please consider the following factors when writing the code:
Efficiency: The code should be efficient and optimized for performance.
Readability: The code should be easy to understand and follow, with meaningful variable and function names and proper formatting.
Maintainability: The code should be modular and well-organized, allowing for easy modification and extension in the future.
Security: The code should be free of potential security vulnerabilities.
Best practices: The code should follow established best practices for the chosen programming language and framework.
Additionally, please provide comments within the code to explain its functionality and logic.
I look forward to reviewing the code you generate!
Example:
Desired functionality: Write a Python function that takes a list of integers as input and returns the sum of all even numbers in the list.
Considerations:
Programming language: Python
Efficiency: Use an efficient algorithm for iterating through the list and checking for even numbers.
Readability: Use meaningful variable names and comments to explain the logic.
Maintainability: The function should be self-contained and easy to modify.
Just respond with only the code, DO NOT give any explaination or comments.
""")

In [77]:
reviewer = PromptTemplate.from_template("""
Critique the following code and identify any errors. Suggest how to fix the errors and improve the code.

CODE:
{code}
Please provide the following information in your response:
Programming language: Identify the programming language of the code.
Functionality: Briefly describe what the code is intended to do.
Errors: List any errors you find in the code, including syntax errors, logic errors, and potential runtime errors.
Fixes: Suggest specific fixes for the identified errors.
Improvements: Recommend ways to improve the code's readability, efficiency, or maintainability.
Example:
def sum(a b):
  return a + b
Use code with caution.
Critique:
Programming language: Python
Functionality: This code defines a function called sum that takes two arguments and returns their sum.
Errors:
Syntax error: Missing comma between arguments in the function definition.
Fixes:
Add a comma between a and b in the function definition: def sum(a, b):
Improvements:
Add type hints to the function arguments and return value for better readability and type checking.
Revised code:
def sum(a: int, b: int) -> int:
  return a + b
""")

In [71]:
refiner = PromptTemplate.from_template("""
Apply the below suggestion to improve the code.
{code}
{suggestion}
""")

## Chains

In [78]:
# Setup the chains
from langchain.schema import StrOutputParser

output_parser = StrOutputParser()

programmer = programmer | llm | output_parser
reviewer = reviewer | llm | output_parser
refiner = refiner | llm | output_parser

# Input

In [82]:
result = programmer.invoke({"Task":"Generate the nth Fib number"})
print(result)

**Understanding the Fibonacci Sequence**:

The Fibonacci sequence is a series of numbers where each number is the sum of the two preceding ones, typically starting with 0 and 1. The sequence goes like this:

```
0, 1, 1, 2, 3, 5, 8, 13, 21, 34, ...
```

**Recursive Approach**:

The `fib` function calculates the nth Fibonacci number using a recursive approach. Here's how it works:

1. **Base Cases**: The function checks if `n` is less than or equal to 1. If it is, it returns `n` itself. This is because the first two Fibonacci numbers are 0 and 1.

2. **Recursive Calls**: If `n` is greater than 1, the function makes two recursive calls:
   - `fib(n-1)`: This calculates the (n-1)th Fibonacci number.
   - `fib(n-2)`: This calculates the (n-2)th Fibonacci number.

3. **Summation**: The function then adds the results of the two recursive calls. This effectively calculates the nth Fibonacci number by summing the (n-1)th and (n-2)th Fibonacci numbers.

**Example**:

Let's say we want to calcul

# Critic

In [83]:
critic = reviewer.invoke({"code":result})
to_markdown(critic)

> **Programming Language:** Python
> 
> **Functionality:** The code defines a function called `fib` that calculates the nth Fibonacci number using a recursive approach.
> 
> **Errors:**
> 
> 1. **Syntax Error:** The `fib` function is missing a colon (`:`) at the end of its definition.
> 
> 2. **Logic Error:** The base cases in the function are incorrect. The first Fibonacci number is 0, not 1.
> 
> 3. **Potential Runtime Error:** The recursive approach can lead to exponential time complexity for large values of `n`, resulting in a stack overflow error.
> 
> **Fixes:**
> 
> 1. Add a colon at the end of the `fib` function definition.
> 2. Update the base cases to:
>    ```
>    if n <= 0:
>        return n
>    elif n == 1:
>        return 1
>    ```
> 3. Consider using a more efficient approach for large values of `n`, such as memoization or matrix exponentiation.
> 
> **Improvements:**
> 
> 1. **Readability:** Add comments to explain the purpose of the function and its arguments.
> 2. **Efficiency:** Replace the recursive approach with an iterative approach or use memoization to avoid redundant calculations.
> 3. **Maintainability:** Add type annotations to the function arguments and return value for better code readability and type checking.
> 
> **Revised Code:**
> 
> ```python
> def fib(n: int) -> int:
>     """
>     Calculates the nth Fibonacci number using an iterative approach.
> 
>     Args:
>         n (int): The index of the Fibonacci number to calculate.
> 
>     Returns:
>         int: The nth Fibonacci number.
>     """
> 
>     fib_sequence = [0, 1]
> 
>     while len(fib_sequence) <= n:
>         next_fib = fib_sequence[-1] + fib_sequence[-2]
>         fib_sequence.append(next_fib)
> 
>     return fib_sequence[n]
> ```

# Refine

In [84]:
code = refiner.invoke({"code":result, "suggestion":critic})
to_markdown(code)

> **Improved Code with Additional Comments and Type Hints:**
> 
> ```python
> from typing import List
> 
> def fib(n: int) -> int:
>     """
>     Calculates the nth Fibonacci number using an iterative approach.
> 
>     Args:
>         n (int): The index of the Fibonacci number to calculate.
> 
>     Returns:
>         int: The nth Fibonacci number.
>     """
> 
>     # Base cases:
>     if n <= 0:
>         return 0
>     elif n == 1:
>         return 1
> 
>     # Initialize the Fibonacci sequence with the first two numbers.
>     fib_sequence: List[int] = [0, 1]
> 
>     # Iterate until the Fibonacci sequence has at least n+1 elements.
>     while len(fib_sequence) <= n:
>         # Calculate the next Fibonacci number by adding the last two numbers in the sequence.
>         next_fib = fib_sequence[-1] + fib_sequence[-2]
>         # Append the next Fibonacci number to the sequence.
>         fib_sequence.append(next_fib)
> 
>     # Return the nth Fibonacci number.
>     return fib_sequence[n]
> ```
> 
> **Additional Comments and Type Hints:**
> 
> 1. Added type hints to the function parameters and return value.
> 2. Changed the type of the `fib_sequence` variable to `List[int]` to make it clear that it is a list of integers.
> 3. Added comments to explain the purpose of the `fib_sequence` variable.

# Loop

In [ ]:
while True:
    result = programmer.invoke({"Task":"Find the cube root of a number."})
    to_markdown(result)
    critic = reviewer.invoke({"code":result})
    to_markdown(critic)
    code = refiner.invoke({"code":result, "suggestion":critic})
    to_markdown(code)
    # decide when to break :D